In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --no-cache-dir --force-reinstall
import os
os.kill(os.getpid(), 9)  # Reinicia o kernel


In [ ]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 40.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade numpy gensim --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: smart-open
    Found existing installatio

In [ ]:
!pip install lazypredict

In [ ]:
import os

# Manipulação de dados
import pandas as pd
import numpy as np

# RDKit
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

# Gensim
from gensim.models import Word2Vec

# Scikit-learn
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import NuSVC
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# XGBoost
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# LazyPredict
from lazypredict.Supervised import LazyClassifier

# Utilitários
from itertools import product


Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7be4d0534a40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


# dataset juntos

In [ ]:
# Exemplo: lendo um CSV com uma coluna chamada 'smiles'
df = pd.read_csv("/content/df_final.csv")
smiles_list = df['SMILES'].tolist()

## separando usando RDKit

In [ ]:
def smiles_to_tokens(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return []

    tokens = []

    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        idx = atom.GetIdx()
        neighbors = [nbr.GetSymbol() for nbr in atom.GetNeighbors()]
        token = f'{symbol}({",".join(neighbors)})'
        tokens.append(token)

    return tokens

# Exemplo
tokenized = [smiles_to_tokens(s) for s in smiles_list]

[20:41:52] WARNING: not removing hydrogen atom without neighbors
[20:41:52] WARNING: not removing hydrogen atom without neighbors
[20:41:52] WARNING: not removing hydrogen atom without neighbors
[20:41:52] WARNING: not removing hydrogen atom without neighbors
[20:41:52] WARNING: not removing hydrogen atom without neighbors
[20:41:52] WARNING: not removing hydrogen atom without neighbors


## tokenizando usando Word2Vec

In [ ]:
# Treinando o modelo Word2Vec nos tokens
model = Word2Vec(
    sentences=tokenized,
    vector_size=128,
    window=5,
    sg=1,  # skip-gram
    min_count=1,
    epochs=30,
    workers=4
)

# Função para média + soma + máximo
def get_mol_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if not vecs:
        return np.zeros(model.vector_size * 3)
    vecs = np.array(vecs)
    return np.concatenate([vecs.mean(axis=0), vecs.sum(axis=0), vecs.max(axis=0)])

# Aplicando para todas as moléculas
mol_vectors = np.array([get_mol_vector(tokens, model) for tokens in tokenized])
df_ = pd.DataFrame(mol_vectors)

In [ ]:
le = LabelEncoder()
X = df_
y = le.fit_transform(df['Results'])

In [ ]:
# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar o StratifiedKFold (10 folds)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Criar dicionário para armazenar os resultados
results = []

# Iterar sobre os folds
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializar o LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Treinar e testar os modelos
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Salvar os resultados do fold
    results.append(models)

predict_results_both = (pd.concat(results).groupby(level=0).agg(['mean', 'std']).sort_values(by=('F1 Score', 'mean'), ascending=False)
)


 97%|█████████▋| 31/32 [01:28<00:02,  2.03s/it]

[LightGBM] [Info] Number of positive: 1788, number of negative: 1709
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020174 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 79848
[LightGBM] [Info] Number of data points in the train set: 3497, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511295 -> initscore=0.045189
[LightGBM] [Info] Start training from score 0.045189


 97%|█████████▋| 31/32 [01:13<00:02,  2.05s/it]

[LightGBM] [Info] Number of positive: 1788, number of negative: 1709
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 79894
[LightGBM] [Info] Number of data points in the train set: 3497, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511295 -> initscore=0.045189
[LightGBM] [Info] Start training from score 0.045189


 97%|█████████▋| 31/32 [01:27<00:01,  1.93s/it]

...

 97%|█████████▋| 31/32 [01:17<00:01,  1.99s/it]

[LightGBM] [Info] Number of positive: 1789, number of negative: 1709
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 80002
[LightGBM] [Info] Number of data points in the train set: 3498, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511435 -> initscore=0.045748
[LightGBM] [Info] Start training from score 0.045748


 97%|█████████▋| 31/32 [01:09<00:01,  1.94s/it]

[LightGBM] [Info] Number of positive: 1788, number of negative: 1710
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019612 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 80286
[LightGBM] [Info] Number of data points in the train set: 3498, number of used features: 384
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.511149 -> initscore=0.044604
[LightGBM] [Info] Start training from score 0.044604


100%|██████████| 32/32 [01:14<00:00,  2.34s/it]


In [ ]:
predict_results_both.to_csv("results_both_Vector_lazy.csv")

In [ ]:
predict_results_both

Accuracy      Balanced Accuracy      ROC AUC  \
                                  mean  std              mean  std    mean   
Model                                                                        
RandomForestClassifier            0.74 0.02              0.74 0.02    0.74   
ExtraTreesClassifier              0.73 0.02              0.73 0.02    0.73   
LGBMClassifier                    0.73 0.02              0.73 0.02    0.73   
NuSVC                             0.73 0.02              0.73 0.02    0.73   
XGBClassifier                     0.73 0.02              0.73 0.02    0.73   
SVC                               0.72 0.01              0.72 0.01    0.72   
BaggingClassifier                 0.71 0.01              0.71 0.01    0.71   
KNeighborsClassifier              0.70 0.02              0.70 0.02    0.70   
CalibratedClassifierCV            0.70 0.02              0.69 0.02    0.69   
LinearDiscriminantAnalysis        0.69 0.02              0.69 0.02    0.69   
LinearSVC                         0.69 0.02              0.69 0.02    0.69   
RidgeClassifierCV                 0.69 0.02              0.69 0.02    0.69   
RidgeClassifier                   0.69 0.02              0.69 0.02    0.69   
LogisticRegression                0.69 0.02              0.69 0.02    0.69   
AdaBoostClassifier                0.68 0.02              0.68 0.02    0.68   
QuadraticDiscriminantAnalysis     0.68 0.02              0.67 0.03    0.67   
DecisionTreeClassifier            0.67 0.02              0.67 0.02    0.67   
ExtraTreeClassifier               0.66 0.02              0.66 0.02    0.66   
SGDClassifier                     0.63 0.04              0.63 0.04    0.63   
PassiveAggressiveClassifier       0.62 0.02              0.62 0.02    0.62   
Perceptron                        0.62 0.02              0.62 0.02    0.62   
BernoulliNB                       0.62 0.03              0.62 0.03    0.62   
NearestCentroid                   0.61 0.04              0.61 0.04    0.61   
GaussianNB                        0.61 0.02              0.60 0.02    0.60   
LabelSpreading                    0.59 0.01              0.60 0.01    0.60   
LabelPropagation                  0.59 0.01              0.60 0.01    0.60   
DummyClassifier                   0.51 0.00              0.50 0.00    0.50   

                                   F1 Score      Time Taken       
                               std     mean  std       mean  std  
Model                                                             
RandomForestClassifier        0.02     0.74 0.02       6.36 0.23  
ExtraTreesClassifier          0.02     0.73 0.02       1.69 0.29  
LGBMClassifier                0.02     0.73 0.02       5.06 0.35  
NuSVC                         0.02     0.73 0.02       2.53 0.29  
XGBClassifier                 0.02     0.73 0.02       9.32 0.41  
SVC                           0.01     0.72 0.01       2.13 0.14  
BaggingClassifier             0.01     0.71 0.01      12.61 0.56  
KNeighborsClassifier          0.02     0.70 0.02       0.11 0.03  
CalibratedClassifierCV        0.02     0.69 0.02      21.71 3.89  
LinearDiscriminantAnalysis    0.02     0.69 0.02       0.27 0.01  
LinearSVC                     0.02     0.69 0.02       5.52 1.15  
RidgeClassifierCV             0.02     0.69 0.02       0.65 0.37  
RidgeClassifier               0.02     0.69 0.02       0.20 0.15  
LogisticRegression            0.02     0.69 0.02       0.26 0.06  
AdaBoostClassifier            0.02     0.68 0.02      10.46 2.96  
QuadraticDiscriminantAnalysis 0.03     0.67 0.03       0.44 0.12  
DecisionTreeClassifier        0.02     0.67 0.02       2.00 0.22  
ExtraTreeClassifier           0.02     0.66 0.02       0.08 0.02  
SGDClassifier                 0.04     0.63 0.04       0.47 0.12  
PassiveAggressiveClassifier   0.02     0.62 0.02       0.15 0.04  
Perceptron                    0.02     0.62 0.02       0.15 0.07  
BernoulliNB                   0.03     0.61 0.04       0.09 0.00  
NearestCentroi

## GridSearch

In [ ]:
# Parâmetros para o Word2Vec
param_grid_w2v = {
    'vector_size': [50 ,128, 160, 256],  # mais relevante
    'window': [3, 5],                # afeta coocorrência local
    'sg': [1],                       # skip-gram tende a ser melhor com poucos dados
    'epochs': [20],                 # manter fixo se não houver indício de underfitting
    'min_count': [1]                # garantir que tokens raros sejam mantidos
}

# Classificadores e seus grids
modelos = {
    "RandomForest": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", RandomForestClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100, 200],
            "model__max_depth": [3, 6, 9]
        }
    },
    "LightGBM": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LGBMClassifier(random_state=42, verbose=-1))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100, 200],
            "model__max_depth": [3, 6, 9]
        }
    },
    "ExtraTrees": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", ExtraTreesClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100, 200],
            "model__max_depth": [3, 6, 9]
        }
    }
}

# Scorers
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Caminho para salvar resultados
caminho_csv = "resultados_gridsearch_w2v_conjunto.csv"
primeira_execucao = not os.path.exists(caminho_csv)

# Carregar CSV existente para evitar repetições
if os.path.exists(caminho_csv):
    df_existente = pd.read_csv(caminho_csv)
else:
    df_existente = pd.DataFrame()

# Combinações de hiperparâmetros do Word2Vec
combinacoes_w2v = list(product(
    param_grid_w2v['vector_size'],
    param_grid_w2v['window'],
    param_grid_w2v['epochs'],
    param_grid_w2v['sg'],
    param_grid_w2v['min_count']
))

for size, window, epochs, sg, min_count in combinacoes_w2v:
    print(f"\n🧠 Word2Vec: size={size}, window={window}, epochs={epochs}, sg={sg}, min_count={min_count}")
    try:
        # Treinamento do modelo Word2Vec
        w2v_model = Word2Vec(
            sentences=tokenized,
            vector_size=size,
            window=window,
            sg=sg,
            min_count=min_count,
            epochs=epochs,
            workers=4
        )

        def get_mol_vector(tokens, model):
            vecs = [model.wv[t] for t in tokens if t in model.wv]
            return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

        X = np.array([get_mol_vector(tokens, w2v_model) for tokens in tokenized])

        # Avaliar cada classificador
        for nome_modelo, config in modelos.items():
            print(f"  🔍 Classificando com: {nome_modelo}")

            # Verificar se já foi processado
            if not df_existente.empty:
                filtro = (
                    (df_existente['modelo'] == nome_modelo) &
                    (df_existente['vector_size'] == size) &
                    (df_existente['window'] == window) &
                    (df_existente['epochs'] == epochs) &
                    (df_existente['sg'] == sg) &
                    (df_existente['min_count'] == min_count)
                )
                if filtro.any():
                    print(f"    ⚠️ Já processado anteriormente. Pulando...\n")
                    continue

            grid = GridSearchCV(
                estimator=config["pipeline"],
                param_grid=config["param_grid"],
                scoring=scoring,
                refit="f1",
                cv=cv,
                verbose=0,
                n_jobs=-1,
                return_train_score=False
            )

            grid.fit(X, y)

            df_resultado = pd.DataFrame(grid.cv_results_)
            df_resultado['modelo'] = nome_modelo
            df_resultado['vector_size'] = size
            df_resultado['window'] = window
            df_resultado['epochs'] = epochs
            df_resultado['sg'] = sg
            df_resultado['min_count'] = min_count

            # Filtrar colunas relevantes
            colunas_mostrar = [
                'modelo', 'vector_size', 'window', 'epochs', 'sg', 'min_count',
                'mean_test_accuracy', 'std_test_accuracy',
                'mean_test_precision', 'std_test_precision',
                'mean_test_recall', 'std_test_recall',
                'mean_test_f1', 'std_test_f1',
                'mean_test_roc_auc', 'std_test_roc_auc',
                'params'
            ]

            df_filtrado = df_resultado[colunas_mostrar]

            # Salvar incrementalmente
            df_filtrado.to_csv(caminho_csv, mode='a', header=primeira_execucao, index=False)
            primeira_execucao = False

            print(f"    ✅ Melhor F1: {grid.best_score_:.4f} | Parâmetros: {grid.best_params_}")

    except Exception as e:
        print("⚠️ Erro ao treinar com esta combinação:", e)

print("\n✅ GridSearch finalizado para todas as combinações! Resultados salvos em:", caminho_csv)


🧠 Word2Vec: size=50, window=3, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.7086 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 200}
  🔍 Classificando com: LightGBM
    ✅ Melhor F1: 0.7139 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 200}
  🔍 Classificando com: ExtraTrees
    ✅ Melhor F1: 0.6898 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 50}

🧠 Word2Vec: size=50, window=5, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.7138 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 100}
  🔍 Classificando com: LightGBM
    ✅ Melhor F1: 0.7178 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 200}
  🔍 Classificando com: ExtraTrees
    ✅ Melhor F1: 0.6891 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 50}

🧠 Word2Vec: size=128, window=3, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.7105 | Parâmetros: 

In [ ]:
df_resultados = pd.read_csv(caminho_csv)
df_resultados.sort_values(by=['mean_test_f1', 'mean_test_accuracy'], ascending=False)

,modelo,vector_size,window,epochs,sg,min_count,mean_test_accuracy,std_test_accuracy,mean_test_precision,std_test_precision,mean_test_recall,std_test_recall,mean_test_f1,std_test_f1,mean_test_roc_auc,std_test_roc_auc,params
125,LightGBM,160,3,20,1,1,0.73,0.01,0.74,0.02,0.72,0.02,0.73,0.01,0.80,0.01,"{'model__max_depth': 9, 'model__n_estimators':..."
176,LightGBM,256,3,20,1,1,0.73,0.02,0.74,0.02,0.72,0.01,0.73,0.01,0.79,0.02,"{'model__max_depth': 6, 'model__n_estimators':..."
70,LightGBM,128,3,20,1,1,0.72,0.02,0.73,0.02,0.72,0.02,0.73,0.02,0.79,0.02,"{'model__max_depth': 9, 'model__n_estimators':..."
94,LightGBM,128,5,20,1,1,0.72,0.02,0.73,0.02,0.72,0.03,0.73,0.02,0.79,0.02,"{'model__max_depth': 6, 'model__n_estimators':..."
203,LightGBM,256,5,20,1,1,0.72,0.01,0.74,0.02,0.71,0.02,0.72,0.01,0.80,0.01,"{'model__max_depth': 6, 'model__n_estimators':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,RandomForest,50,5,20,1,1,0.66,0.02,0.70,0.03,0.61,0.04,0.65,0.03,0.72,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."
1,RandomForest,50,3,20,1,1,0.66,0.03,0.69,0.03,0.61,0.04,0.65,0.03,0.71,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."
27,RandomForest,50,5,20,1,1,0.66,0.02,0.70,0.02,0.61,0.05,0.65,0.03,0.72,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."
2,RandomForest,50,3,20,1,1,0.66,0.02,0.69,0.02,0.61,0.04,0.65,0.03,0.72,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."


### melhor resultado

In [ ]:
# Melhor configuração do Word2Vec
best_w2v_config = {
    'vector_size': 128,
    'window': 5,
    'epochs': 20,
    'sg': 1,
    'min_count': 1
}

# Treinar o modelo Word2Vec
print("🧠 Treinando Word2Vec com melhor configuração...")
w2v_model = Word2Vec(
    sentences=tokenized,
    vector_size=best_w2v_config['vector_size'],
    window=best_w2v_config['window'],
    sg=best_w2v_config['sg'],
    min_count=best_w2v_config['min_count'],
    epochs=best_w2v_config['epochs'],
    workers=4
)

# Função para gerar vetor da molécula
def get_mol_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

X = np.array([get_mol_vector(tokens, w2v_model) for tokens in tokenized])

# Pipeline com LightGBM
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LGBMClassifier(
        max_depth=6,
        n_estimators=200,
        random_state=42,
        verbose=-1
    ))
])

# Validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Scorers para ambas as classes
scoring = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc',

    'precision_pos1': make_scorer(precision_score, average='binary', pos_label=1),
    'recall_pos1': make_scorer(recall_score, average='binary', pos_label=1),
    'f1_pos1': make_scorer(f1_score, average='binary', pos_label=1),

    'precision_pos0': make_scorer(precision_score, average='binary', pos_label=0),
    'recall_pos0': make_scorer(recall_score, average='binary', pos_label=0),
    'f1_pos0': make_scorer(f1_score, average='binary', pos_label=0),
}

# Avaliação
print("\n🚀 Avaliando modelo com validação cruzada (10 folds)...")
resultados = cross_validate(
    pipeline, X, y,
    scoring=scoring,
    cv=cv,
    return_train_score=False
)

# Impressão dos resultados
print("\n🔍 Resultados médios na validação cruzada:\n")

# Separando por classe
for label in ['pos1', 'pos0']:
    print(f"📊 Classe positiva: y = {label[-1]}")
    for metric in ['precision', 'recall', 'f1']:
        key = f'test_{metric}_{label}'
        mean = resultados[key].mean()
        std = resultados[key].std()
        print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")
    print()

# Accuracy e AUC gerais
for metric in ['accuracy', 'roc_auc']:
    mean = resultados[f'test_{metric}'].mean()
    std = resultados[f'test_{metric}'].std()
    print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")

🧠 Treinando Word2Vec com melhor configuração...

🚀 Avaliando modelo com validação cruzada (10 folds)...

🔍 Resultados médios na validação cruzada:

📊 Classe positiva: y = 1
precision : 0.7328 ± 0.0215
recall    : 0.7202 ± 0.0172
f1        : 0.7261 ± 0.0111

📊 Classe positiva: y = 0
precision : 0.7121 ± 0.0116
recall    : 0.7241 ± 0.0341
f1        : 0.7177 ± 0.0195

accuracy  : 0.7221 ± 0.0142
roc_auc   : 0.7961 ± 0.0146


# dataset separados


## in vivo

In [ ]:
# Exemplo: lendo um CSV com uma coluna chamada 'smiles'
df = pd.read_csv("/content/df_final_vivo.csv")
smiles_list = df['SMILES'].tolist()

In [ ]:
def smiles_to_tokens(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return []

    tokens = []

    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        idx = atom.GetIdx()
        neighbors = [nbr.GetSymbol() for nbr in atom.GetNeighbors()]
        token = f'{symbol}({",".join(neighbors)})'
        tokens.append(token)

    return tokens

# Exemplo
tokenized = [smiles_to_tokens(s) for s in smiles_list]

[21:46:29] WARNING: not removing hydrogen atom without neighbors
[21:46:29] WARNING: not removing hydrogen atom without neighbors


In [ ]:
# Treinando o modelo Word2Vec nos tokens
model = Word2Vec(
    sentences=tokenized,
    vector_size=128,
    window=5,
    sg=1,  # skip-gram
    min_count=1,
    epochs=30,
    workers=4
)


# Função para média + soma + máximo
def get_mol_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if not vecs:
        return np.zeros(model.vector_size * 3)
    vecs = np.array(vecs)
    return np.concatenate([vecs.mean(axis=0), vecs.sum(axis=0), vecs.max(axis=0)])

# Aplicando para todas as moléculas
mol_vectors = np.array([get_mol_vector(tokens, model) for tokens in tokenized])
df_Vivo = pd.DataFrame(mol_vectors)

In [ ]:
le = LabelEncoder()
X = df_Vivo
y = le.fit_transform(df['Results'])


In [ ]:
# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar o StratifiedKFold (10 folds)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Criar dicionário para armazenar os resultados
results = []

# Iterar sobre os folds
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializar o LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Treinar e testar os modelos
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Salvar os resultados do fold
    results.append(models)

predict_results_vivo = (pd.concat(results).groupby(level=0).agg(['mean', 'std']).sort_values(by=('F1 Score', 'mean'), ascending=False)
)


100%|██████████| 32/32 [00:42<00:00,  1.32s/it]


In [ ]:
predict_results_vivo.to_csv('predict_results_vivo_lazy.csv')

In [ ]:
predict_results_vivo

Accuracy      Balanced Accuracy      ROC AUC  \
                                  mean  std              mean  std    mean   
Model                                                                        
ExtraTreesClassifier              0.80 0.02              0.77 0.02    0.77   
RandomForestClassifier            0.80 0.02              0.77 0.03    0.77   
XGBClassifier                     0.79 0.02              0.77 0.02    0.77   
LGBMClassifier                    0.79 0.02              0.76 0.02    0.76   
NuSVC                             0.78 0.02              0.76 0.02    0.76   
BaggingClassifier                 0.77 0.02              0.74 0.02    0.74   
KNeighborsClassifier              0.77 0.03              0.75 0.03    0.75   
SVC                               0.77 0.02              0.73 0.02    0.73   
RidgeClassifier                   0.74 0.03              0.72 0.03    0.72   
RidgeClassifierCV                 0.74 0.03              0.71 0.03    0.71   
LinearSVC                         0.74 0.03              0.71 0.03    0.71   
LogisticRegression                0.73 0.03              0.71 0.03    0.71   
QuadraticDiscriminantAnalysis     0.73 0.02              0.71 0.02    0.71   
AdaBoostClassifier                0.74 0.03              0.70 0.04    0.70   
LinearDiscriminantAnalysis        0.73 0.02              0.71 0.03    0.71   
DecisionTreeClassifier            0.71 0.04              0.70 0.04    0.70   
ExtraTreeClassifier               0.70 0.04              0.69 0.03    0.69   
CalibratedClassifierCV            0.72 0.03              0.66 0.04    0.66   
GaussianNB                        0.69 0.03              0.66 0.04    0.66   
SGDClassifier                     0.67 0.03              0.66 0.04    0.66   
PassiveAggressiveClassifier       0.66 0.04              0.64 0.04    0.64   
BernoulliNB                       0.66 0.05              0.63 0.06    0.63   
Perceptron                        0.65 0.05              0.63 0.06    0.63   
LabelSpreading                    0.69 0.01              0.61 0.02    0.61   
LabelPropagation                  0.69 0.01              0.61 0.01    0.61   
NearestCentroid                   0.63 0.05              0.60 0.05    0.60   
DummyClassifier                   0.62 0.00              0.50 0.00    0.50   

                                   F1 Score      Time Taken       
                               std     mean  std       mean  std  
Model                                                             
ExtraTreesClassifier          0.02     0.80 0.02       0.93 0.22  
RandomForestClassifier        0.03     0.79 0.02       3.51 0.26  
XGBClassifier                 0.02     0.79 0.02       6.04 0.65  
LGBMClassifier                0.02     0.78 0.02       4.34 0.37  
NuSVC                         0.02     0.78 0.02       0.73 0.10  
BaggingClassifier             0.02     0.77 0.02       6.97 0.23  
KNeighborsClassifier          0.03     0.76 0.03       0.06 0.01  
SVC                           0.02     0.76 0.02       0.69 0.11  
RidgeClassifier               0.03     0.74 0.03       0.12 0.07  
RidgeClassifierCV             0.03     0.74 0.03       0.39 0.23  
LinearSVC                     0.03     0.74 0.03       2.13 0.96  
LogisticRegression            0.03     0.73 0.03       0.19 0.06  
QuadraticDiscriminantAnalysis 0.02     0.73 0.02       0.31 0.11  
AdaBoostClassifier            0.04     0.73 0.03       5.33 0.33  
LinearDiscriminantAnalysis    0.03     0.72 0.02       0.22 0.05  
DecisionTreeClassifier        0.04     0.71 0.04       1.12 0.11  
ExtraTreeClassifier           0.03     0.70 0.03       0.05 0.01  
CalibratedClassifierCV        0.04     0.70 0.04       7.10 0.95  
GaussianNB                    0.04     0.68 0.04       0.05 0.01  
SGDClassifier                 0.04     0.67 0.03       0.25 0.05  
PassiveAggressiveClassifier   0.04     0.66 0.04       0.10 0.02  
BernoulliNB                   0.06     0.66 0.05       0.07 0.02  
Perceptron    

### gridsearch

In [ ]:
# Parâmetros para o Word2Vec
param_grid_w2v = {
    'vector_size': [50 ,128, 160, 256],  # mais relevante
    'window': [3, 5],                # afeta coocorrência local
    'sg': [1],                       # skip-gram tende a ser melhor com poucos dados
    'epochs': [20],                 # manter fixo se não houver indício de underfitting
    'min_count': [1]
}

# Classificadores e seus grids
modelos = {
    "RandomForest": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", RandomForestClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "LightGBM": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LGBMClassifier(random_state=42, verbose=-1))  # verbose corrigido
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100, 200],
            "model__max_depth": [3, 6, 9],
            #"model__learning_rate": [0.005, 0.01, 0.05, 0.1],
            #"model__num_leaves": [15, 31, 63]
        }
    },
    "ExtraTrees": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", ExtraTreesClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    }
}

# Scorers
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Caminho para salvar resultados
caminho_csv = "resultados_gridsearch_w2v_vivo.csv"
primeira_execucao = not os.path.exists(caminho_csv)

# Carregar CSV existente para evitar repetições
if os.path.exists(caminho_csv):
    df_existente = pd.read_csv(caminho_csv)
else:
    df_existente = pd.DataFrame()

# Combinações de hiperparâmetros do Word2Vec
combinacoes_w2v = list(product(
    param_grid_w2v['vector_size'],
    param_grid_w2v['window'],
    param_grid_w2v['epochs'],
    param_grid_w2v['sg'],
    param_grid_w2v['min_count']
))

for size, window, epochs, sg, min_count in combinacoes_w2v:
    print(f"\n🧠 Word2Vec: size={size}, window={window}, epochs={epochs}, sg={sg}, min_count={min_count}")
    try:
        # Treinamento do modelo Word2Vec
        w2v_model = Word2Vec(
            sentences=tokenized,
            vector_size=size,
            window=window,
            sg=sg,
            min_count=min_count,
            epochs=epochs,
            workers=4
        )

        def get_mol_vector(tokens, model):
            vecs = [model.wv[t] for t in tokens if t in model.wv]
            return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

        X = np.array([get_mol_vector(tokens, w2v_model) for tokens in tokenized])

        # Avaliar cada classificador
        for nome_modelo, config in modelos.items():
            print(f"  🔍 Classificando com: {nome_modelo}")

            # Verificar se já foi processado
            if not df_existente.empty:
                filtro = (
                    (df_existente['modelo'] == nome_modelo) &
                    (df_existente['vector_size'] == size) &
                    (df_existente['window'] == window) &
                    (df_existente['epochs'] == epochs) &
                    (df_existente['sg'] == sg) &
                    (df_existente['min_count'] == min_count)
                )
                if filtro.any():
                    print(f"    ⚠️ Já processado anteriormente. Pulando...\n")
                    continue

            grid = GridSearchCV(
                estimator=config["pipeline"],
                param_grid=config["param_grid"],
                scoring=scoring,
                refit="f1",
                cv=cv,
                verbose=0,
                n_jobs=-1,
                return_train_score=False
            )

            grid.fit(X, y)

            df_resultado = pd.DataFrame(grid.cv_results_)
            df_resultado['modelo'] = nome_modelo
            df_resultado['vector_size'] = size
            df_resultado['window'] = window
            df_resultado['epochs'] = epochs
            df_resultado['sg'] = sg
            df_resultado['min_count'] = min_count

            # Filtrar colunas relevantes
            colunas_mostrar = [
                'modelo', 'vector_size', 'window', 'epochs', 'sg', 'min_count',
                'mean_test_accuracy', 'mean_test_precision', 'mean_test_recall',
                'mean_test_f1', 'mean_test_roc_auc', 'params'
            ]

            df_filtrado = df_resultado[colunas_mostrar]

            # Salvar incrementalmente
            df_filtrado.to_csv(caminho_csv, mode='a', header=primeira_execucao, index=False)
            primeira_execucao = False

            print(f"    ✅ Melhor F1: {grid.best_score_:.4f} | Parâmetros: {grid.best_params_}")

    except Exception as e:
        print("⚠️ Erro ao treinar com esta combinação:", e)

print("\n✅ GridSearch finalizado para todas as combinações! Resultados salvos em:", caminho_csv)


🧠 Word2Vec: size=50, window=3, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.5642 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}
  🔍 Classificando com: LightGBM
    ✅ Melhor F1: 0.6774 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 200}
  🔍 Classificando com: ExtraTrees
    ✅ Melhor F1: 0.4135 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🧠 Word2Vec: size=50, window=5, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.5734 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 100}
  🔍 Classificando com: LightGBM
    ✅ Melhor F1: 0.6798 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 50}
  🔍 Classificando com: ExtraTrees
    ✅ Melhor F1: 0.4284 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🧠 Word2Vec: size=128, window=3, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.5744 | Parâmetros: {'

In [ ]:
df_resultados = pd.read_csv(caminho_csv)
df_resultados.sort_values(by=['mean_test_f1', 'mean_test_accuracy'], ascending=False)

,modelo,vector_size,window,epochs,sg,min_count,mean_test_accuracy,mean_test_precision,mean_test_recall,mean_test_f1,mean_test_roc_auc,params
111,LightGBM,256,3,20,1,1,0.79,0.76,0.65,0.70,0.83,"{'model__max_depth': 6, 'model__n_estimators':..."
131,LightGBM,256,5,20,1,1,0.79,0.76,0.65,0.70,0.84,"{'model__max_depth': 9, 'model__n_estimators':..."
113,LightGBM,256,3,20,1,1,0.79,0.77,0.64,0.70,0.84,"{'model__max_depth': 9, 'model__n_estimators':..."
79,LightGBM,160,3,20,1,1,0.79,0.76,0.64,0.70,0.83,"{'model__max_depth': 9, 'model__n_estimators':..."
110,LightGBM,256,3,20,1,1,0.79,0.76,0.64,0.70,0.84,"{'model__max_depth': 6, 'model__n_estimators':..."
...,...,...,...,...,...,...,...,...,...,...,...,...
48,ExtraTrees,128,3,20,1,1,0.66,0.84,0.12,0.21,0.73,"{'model__max_depth': 3, 'model__n_estimators':..."
14,ExtraTrees,50,3,20,1,1,0.65,0.87,0.10,0.18,0.72,"{'model__max_depth': 3, 'model__n_estimators':..."
13,ExtraTrees,50,3,20,1,1,0.65,0.87,0.10,0.18,0.72,"{'model__max_depth': 3, 'model__n_estimators':..."
31,ExtraTrees,50,5,20,1,1,0.64,0.86,0.08,0.15,0.73,"{'model__max_depth': 3, 'model__n_estimators':..."


### Melhor resultado

In [ ]:
# Melhor configuração do Word2Vec
best_w2v_config = {
    'vector_size': 160,
    'window': 3,
    'epochs': 20,
    'sg': 1,
    'min_count': 1
}

# Treinar o modelo Word2Vec
print("🧠 Treinando Word2Vec com melhor configuração...")
w2v_model = Word2Vec(
    sentences=tokenized,
    vector_size=best_w2v_config['vector_size'],
    window=best_w2v_config['window'],
    sg=best_w2v_config['sg'],
    min_count=best_w2v_config['min_count'],
    epochs=best_w2v_config['epochs'],
    workers=4
)

# Função para gerar vetor da molécula
def get_mol_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

X = np.array([get_mol_vector(tokens, w2v_model) for tokens in tokenized])

# Pipeline com LightGBM
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LGBMClassifier(
        max_depth=6,
        n_estimators=200,
        random_state=42,
        verbose=-1
    ))
])

# Validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Scorers para ambas as classes
scoring = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc',

    'precision_pos1': make_scorer(precision_score, average='binary', pos_label=1),
    'recall_pos1': make_scorer(recall_score, average='binary', pos_label=1),
    'f1_pos1': make_scorer(f1_score, average='binary', pos_label=1),

    'precision_pos0': make_scorer(precision_score, average='binary', pos_label=0),
    'recall_pos0': make_scorer(recall_score, average='binary', pos_label=0),
    'f1_pos0': make_scorer(f1_score, average='binary', pos_label=0),
}

# Avaliação
print("\n🚀 Avaliando modelo com validação cruzada (10 folds)...")
resultados = cross_validate(
    pipeline, X, y,
    scoring=scoring,
    cv=cv,
    return_train_score=False
)

# Impressão dos resultados
print("\n🔍 Resultados médios na validação cruzada:\n")

# Separando por classe
for label in ['pos1', 'pos0']:
    print(f"📊 Classe positiva: y = {label[-1]}")
    for metric in ['precision', 'recall', 'f1']:
        key = f'test_{metric}_{label}'
        mean = resultados[key].mean()
        std = resultados[key].std()
        print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")
    print()

# Accuracy e AUC gerais
for metric in ['accuracy', 'roc_auc']:
    mean = resultados[f'test_{metric}'].mean()
    std = resultados[f'test_{metric}'].std()
    print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")

🧠 Treinando Word2Vec com melhor configuração...

🚀 Avaliando modelo com validação cruzada (10 folds)...

🔍 Resultados médios na validação cruzada:

📊 Classe positiva: y = 1
precision : 0.7722 ± 0.0425
recall    : 0.6478 ± 0.0402
f1        : 0.7037 ± 0.0332

📊 Classe positiva: y = 0
precision : 0.8032 ± 0.0192
recall    : 0.8816 ± 0.0265
f1        : 0.8404 ± 0.0178

accuracy  : 0.7926 ± 0.0228
roc_auc   : 0.8280 ± 0.0219


## in vitro

In [ ]:
# Exemplo: lendo um CSV com uma coluna chamada 'smiles'
df = pd.read_csv("/content/df_final_vitro.csv")
smiles_list = df['SMILES'].tolist()

In [ ]:
def smiles_to_tokens(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return []

    tokens = []

    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        idx = atom.GetIdx()
        neighbors = [nbr.GetSymbol() for nbr in atom.GetNeighbors()]
        token = f'{symbol}({",".join(neighbors)})'
        tokens.append(token)

    return tokens

# Exemplo
tokenized = [smiles_to_tokens(s) for s in smiles_list]

[22:12:39] WARNING: not removing hydrogen atom without neighbors
[22:12:39] WARNING: not removing hydrogen atom without neighbors
[22:12:39] WARNING: not removing hydrogen atom without neighbors
[22:12:39] WARNING: not removing hydrogen atom without neighbors


In [ ]:
# Treinando o modelo Word2Vec nos tokens
model = Word2Vec(
    sentences=tokenized,
    vector_size=128,
    window=5,
    sg=1,  # skip-gram
    min_count=1,
    epochs=30,
    workers=4
)


# Função para média + soma + máximo
def get_mol_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    if not vecs:
        return np.zeros(model.vector_size * 3)
    vecs = np.array(vecs)
    return np.concatenate([vecs.mean(axis=0), vecs.sum(axis=0), vecs.max(axis=0)])

# Aplicando para todas as moléculas
mol_vectors = np.array([get_mol_vector(tokens, model) for tokens in tokenized])
df_Vivo = pd.DataFrame(mol_vectors)

In [ ]:
le = LabelEncoder()
X = df_Vivo
y = le.fit_transform(df['Results'])

In [ ]:
# Normalizar os dados (opcional, mas recomendado para alguns modelos)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Configurar o StratifiedKFold (10 folds)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Criar dicionário para armazenar os resultados
results = []

# Iterar sobre os folds
for train_idx, test_idx in cv.split(X_scaled, y):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Inicializar o LazyClassifier
    clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

    # Treinar e testar os modelos
    models, predictions = clf.fit(X_train, X_test, y_train, y_test)

    # Salvar os resultados do fold
    results.append(models)

predict_results_vitro = (pd.concat(results).groupby(level=0).agg(['mean', 'std']).sort_values(by=('F1 Score', 'mean'), ascending=False)
)


100%|██████████| 32/32 [01:01<00:00,  1.91s/it]


In [ ]:
predict_results_vitro.to_csv('predict_results_vitro_lazy.csv')

In [ ]:
predict_results_vitro

Accuracy      Balanced Accuracy      ROC AUC  \
                                  mean  std              mean  std    mean   
Model                                                                        
LGBMClassifier                    0.75 0.02              0.73 0.02    0.73   
RandomForestClassifier            0.75 0.03              0.72 0.03    0.72   
ExtraTreesClassifier              0.75 0.02              0.72 0.02    0.72   
NuSVC                             0.74 0.02              0.71 0.02    0.71   
XGBClassifier                     0.74 0.02              0.71 0.02    0.71   
BaggingClassifier                 0.73 0.03              0.72 0.03    0.72   
QuadraticDiscriminantAnalysis     0.72 0.01              0.69 0.02    0.69   
SVC                               0.72 0.02              0.68 0.02    0.68   
LinearSVC                         0.71 0.02              0.68 0.03    0.68   
RidgeClassifier                   0.71 0.02              0.68 0.03    0.68   
LogisticRegression                0.71 0.03              0.68 0.04    0.68   
KNeighborsClassifier              0.71 0.01              0.68 0.02    0.68   
RidgeClassifierCV                 0.71 0.03              0.68 0.03    0.68   
LinearDiscriminantAnalysis        0.70 0.02              0.67 0.02    0.67   
DecisionTreeClassifier            0.69 0.04              0.68 0.04    0.68   
ExtraTreeClassifier               0.69 0.02              0.67 0.02    0.67   
AdaBoostClassifier                0.69 0.03              0.66 0.03    0.66   
CalibratedClassifierCV            0.71 0.02              0.65 0.02    0.65   
Perceptron                        0.66 0.03              0.63 0.03    0.63   
SGDClassifier                     0.66 0.03              0.63 0.04    0.63   
PassiveAggressiveClassifier       0.65 0.04              0.63 0.04    0.63   
GaussianNB                        0.63 0.03              0.59 0.03    0.59   
NearestCentroid                   0.60 0.02              0.59 0.02    0.59   
BernoulliNB                       0.59 0.03              0.58 0.03    0.58   
LabelSpreading                    0.58 0.03              0.65 0.03    0.65   
LabelPropagation                  0.58 0.03              0.65 0.03    0.65   
DummyClassifier                   0.61 0.00              0.50 0.00    0.50   

                                   F1 Score      Time Taken       
                               std     mean  std       mean  std  
Model                                                             
LGBMClassifier                0.02     0.75 0.02       4.58 0.34  
RandomForestClassifier        0.03     0.74 0.03       4.90 0.27  
ExtraTreesClassifier          0.02     0.74 0.02       1.19 0.04  
NuSVC                         0.02     0.73 0.02       1.50 0.07  
XGBClassifier                 0.02     0.73 0.02       7.96 0.58  
BaggingClassifier             0.03     0.73 0.03       9.69 0.35  
QuadraticDiscriminantAnalysis 0.02     0.71 0.01       0.39 0.14  
SVC                           0.02     0.71 0.02       1.30 0.09  
LinearSVC                     0.03     0.71 0.03       3.90 0.99  
RidgeClassifier               0.03     0.70 0.03       0.13 0.05  
LogisticRegression            0.04     0.70 0.03       0.22 0.06  
KNeighborsClassifier          0.02     0.70 0.01       0.08 0.01  
RidgeClassifierCV             0.03     0.70 0.03       0.49 0.43  
LinearDiscriminantAnalysis    0.02     0.70 0.02       0.26 0.06  
DecisionTreeClassifier        0.04     0.69 0.04       1.51 0.14  
ExtraTreeClassifier           0.02     0.69 0.02       0.06 0.01  
AdaBoostClassifier            0.03     0.68 0.03       7.19 0.23  
CalibratedClassifierCV        0.02     0.68 0.02      14.71 5.19  
Perceptron                    0.03     0.66 0.03       0.13 0.03  
SGDClassifier                 0.04     0.65 0.03       0.34 0.05  
PassiveAggressiveClassifier   0.04     0.65 0.04       0.13 0.04  
GaussianNB                    0.03     0.62 0.03       0.05 0.00  
NearestCentroi

### gridsearch

In [ ]:
# Parâmetros para o Word2Vec
param_grid_w2v = {
    'vector_size': [50 ,128, 160, 256],  # mais relevante
    'window': [3, 5],                # afeta coocorrência local
    'sg': [1],                       # skip-gram tende a ser melhor com poucos dados
    'epochs': [20],                 # manter fixo se não houver indício de underfitting
    'min_count': [1]
}

# Classificadores e seus grids
modelos = {
    "RandomForest": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", RandomForestClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    },
    "LightGBM": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", LGBMClassifier(random_state=42, verbose=-1))  # verbose corrigido
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100, 200],
            "model__max_depth": [3, 6, 9]
        }
    },
    "ExtraTrees": {
        "pipeline": Pipeline([
            ("scaler", StandardScaler()),
            ("model", ExtraTreesClassifier(random_state=42))
        ]),
        "param_grid": {
            "model__n_estimators": [50, 100],
            "model__max_depth": [3, 6]
        }
    }
}

# Scorers
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Caminho para salvar resultados
caminho_csv = "resultados_gridsearch_w2v_vitro.csv"
primeira_execucao = not os.path.exists(caminho_csv)

# Carregar CSV existente para evitar repetições
if os.path.exists(caminho_csv):
    df_existente = pd.read_csv(caminho_csv)
else:
    df_existente = pd.DataFrame()

# Combinações de hiperparâmetros do Word2Vec
combinacoes_w2v = list(product(
    param_grid_w2v['vector_size'],
    param_grid_w2v['window'],
    param_grid_w2v['epochs'],
    param_grid_w2v['sg'],
    param_grid_w2v['min_count']
))

for size, window, epochs, sg, min_count in combinacoes_w2v:
    print(f"\n🧠 Word2Vec: size={size}, window={window}, epochs={epochs}, sg={sg}, min_count={min_count}")
    try:
        # Treinamento do modelo Word2Vec
        w2v_model = Word2Vec(
            sentences=tokenized,
            vector_size=size,
            window=window,
            sg=sg,
            min_count=min_count,
            epochs=epochs,
            workers=4
        )

        def get_mol_vector(tokens, model):
            vecs = [model.wv[t] for t in tokens if t in model.wv]
            return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

        X = np.array([get_mol_vector(tokens, w2v_model) for tokens in tokenized])

        # Avaliar cada classificador
        for nome_modelo, config in modelos.items():
            print(f"  🔍 Classificando com: {nome_modelo}")

            # Verificar se já foi processado
            if not df_existente.empty:
                filtro = (
                    (df_existente['modelo'] == nome_modelo) &
                    (df_existente['vector_size'] == size) &
                    (df_existente['window'] == window) &
                    (df_existente['epochs'] == epochs) &
                    (df_existente['sg'] == sg) &
                    (df_existente['min_count'] == min_count)
                )
                if filtro.any():
                    print(f"    ⚠️ Já processado anteriormente. Pulando...\n")
                    continue

            grid = GridSearchCV(
                estimator=config["pipeline"],
                param_grid=config["param_grid"],
                scoring=scoring,
                refit="f1",
                cv=cv,
                verbose=0,
                n_jobs=-1,
                return_train_score=False
            )

            grid.fit(X, y)

            df_resultado = pd.DataFrame(grid.cv_results_)
            df_resultado['modelo'] = nome_modelo
            df_resultado['vector_size'] = size
            df_resultado['window'] = window
            df_resultado['epochs'] = epochs
            df_resultado['sg'] = sg
            df_resultado['min_count'] = min_count

            # Filtrar colunas relevantes
            colunas_mostrar = [
                'modelo', 'vector_size', 'window', 'epochs', 'sg', 'min_count',
                'mean_test_accuracy', 'std_test_accuracy',
                'mean_test_precision', 'std_test_precision',
                'mean_test_recall', 'std_test_recall',
                'mean_test_f1', 'std_test_f1',
                'mean_test_roc_auc', 'std_test_roc_auc',
                'params'
            ]

            df_filtrado = df_resultado[colunas_mostrar]

            # Salvar incrementalmente
            df_filtrado.to_csv(caminho_csv, mode='a', header=primeira_execucao, index=False)
            primeira_execucao = False

            print(f"    ✅ Melhor F1: {grid.best_score_:.4f} | Parâmetros: {grid.best_params_}")

    except Exception as e:
        print("⚠️ Erro ao treinar com esta combinação:", e)

print("\n✅ GridSearch finalizado para todas as combinações! Resultados salvos em:", caminho_csv)


🧠 Word2Vec: size=50, window=3, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.7884 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}
  🔍 Classificando com: LightGBM
    ✅ Melhor F1: 0.7943 | Parâmetros: {'model__max_depth': 9, 'model__n_estimators': 50}
  🔍 Classificando com: ExtraTrees
    ✅ Melhor F1: 0.7697 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}

🧠 Word2Vec: size=50, window=5, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.7881 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 100}
  🔍 Classificando com: LightGBM
    ✅ Melhor F1: 0.7967 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 50}
  🔍 Classificando com: ExtraTrees
    ✅ Melhor F1: 0.7716 | Parâmetros: {'model__max_depth': 6, 'model__n_estimators': 100}

🧠 Word2Vec: size=128, window=3, epochs=20, sg=1, min_count=1
  🔍 Classificando com: RandomForest
    ✅ Melhor F1: 0.7913 | Parâmetros: {'

In [ ]:
df_resultados = pd.read_csv(caminho_csv)
df_resultados.sort_values(by=['mean_test_f1', 'mean_test_accuracy'], ascending=False)

,modelo,vector_size,window,epochs,sg,min_count,mean_test_accuracy,std_test_accuracy,mean_test_precision,std_test_precision,mean_test_recall,std_test_recall,mean_test_f1,std_test_f1,mean_test_roc_auc,std_test_roc_auc,params
45,LightGBM,128,3,20,1,1,0.75,0.02,0.77,0.03,0.83,0.02,0.80,0.02,0.80,0.02,"{'model__max_depth': 9, 'model__n_estimators':..."
77,LightGBM,160,3,20,1,1,0.75,0.02,0.78,0.03,0.82,0.02,0.80,0.02,0.81,0.03,"{'model__max_depth': 6, 'model__n_estimators':..."
80,LightGBM,160,3,20,1,1,0.75,0.02,0.78,0.02,0.82,0.02,0.80,0.01,0.81,0.02,"{'model__max_depth': 9, 'model__n_estimators':..."
76,LightGBM,160,3,20,1,1,0.74,0.02,0.77,0.02,0.83,0.02,0.80,0.02,0.80,0.02,"{'model__max_depth': 6, 'model__n_estimators':..."
41,LightGBM,128,3,20,1,1,0.74,0.02,0.76,0.02,0.84,0.02,0.80,0.02,0.79,0.02,"{'model__max_depth': 6, 'model__n_estimators':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,ExtraTrees,128,3,20,1,1,0.61,0.00,0.61,0.00,1.00,0.00,0.76,0.00,0.72,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."
99,ExtraTrees,160,5,20,1,1,0.61,0.00,0.61,0.00,1.00,0.00,0.76,0.00,0.72,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."
81,ExtraTrees,160,3,20,1,1,0.61,0.00,0.61,0.00,1.00,0.00,0.76,0.00,0.71,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."
116,ExtraTrees,256,3,20,1,1,0.61,0.00,0.61,0.00,1.00,0.00,0.76,0.00,0.72,0.02,"{'model__max_depth': 3, 'model__n_estimators':..."


# Melhor resultado

In [ ]:
# Melhor configuração do Word2Vec
best_w2v_config = {
    'vector_size': 128,
    'window': 5,
    'epochs': 20,
    'sg': 1,
    'min_count': 1
}

# Treinar o modelo Word2Vec
print("🧠 Treinando Word2Vec com melhor configuração...")
w2v_model = Word2Vec(
    sentences=tokenized,
    vector_size=best_w2v_config['vector_size'],
    window=best_w2v_config['window'],
    sg=best_w2v_config['sg'],
    min_count=best_w2v_config['min_count'],
    epochs=best_w2v_config['epochs'],
    workers=4
)

# Função para gerar vetor da molécula
def get_mol_vector(tokens, model):
    vecs = [model.wv[t] for t in tokens if t in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

X = np.array([get_mol_vector(tokens, w2v_model) for tokens in tokenized])

# Pipeline com LightGBM
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LGBMClassifier(
        max_depth=6,
        n_estimators=200,
        random_state=42,
        verbose=-1
    ))
])

# Validação cruzada
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Scorers para ambas as classes
scoring = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc',

    'precision_pos1': make_scorer(precision_score, average='binary', pos_label=1),
    'recall_pos1': make_scorer(recall_score, average='binary', pos_label=1),
    'f1_pos1': make_scorer(f1_score, average='binary', pos_label=1),

    'precision_pos0': make_scorer(precision_score, average='binary', pos_label=0),
    'recall_pos0': make_scorer(recall_score, average='binary', pos_label=0),
    'f1_pos0': make_scorer(f1_score, average='binary', pos_label=0),
}

# Avaliação
print("\n🚀 Avaliando modelo com validação cruzada (10 folds)...")
resultados = cross_validate(
    pipeline, X, y,
    scoring=scoring,
    cv=cv,
    return_train_score=False
)

# Impressão dos resultados
print("\n🔍 Resultados médios na validação cruzada:\n")

# Separando por classe
for label in ['pos1', 'pos0']:
    print(f"📊 Classe positiva: y = {label[-1]}")
    for metric in ['precision', 'recall', 'f1']:
        key = f'test_{metric}_{label}'
        mean = resultados[key].mean()
        std = resultados[key].std()
        print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")
    print()

# Accuracy e AUC gerais
for metric in ['accuracy', 'roc_auc']:
    mean = resultados[f'test_{metric}'].mean()
    std = resultados[f'test_{metric}'].std()
    print(f"{metric:<10}: {mean:.4f} ± {std:.4f}")

🧠 Treinando Word2Vec com melhor configuração...

🚀 Avaliando modelo com validação cruzada (10 folds)...

🔍 Resultados médios na validação cruzada:

📊 Classe positiva: y = 1
precision : 0.7648 ± 0.0167
recall    : 0.8232 ± 0.0201
f1        : 0.7928 ± 0.0152

📊 Classe positiva: y = 0
precision : 0.6811 ± 0.0294
recall    : 0.5979 ± 0.0353
f1        : 0.6364 ± 0.0291

accuracy  : 0.7361 ± 0.0194
roc_auc   : 0.7985 ± 0.0264
